# Decision Tree Regressor using Pyspark MLLib's

Importing necessary packages

In [1]:
from pyspark.sql import SparkSession
from pyspark.mllib.feature import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.sql.functions import col, udf
from math import radians, sin, cos, sqrt, atan2
from pyspark.sql.types import DoubleType

## 1. Preparation

Let's initialize a Spark session:

In [2]:
#pyspark init
builder = SparkSession.builder\
            .appName('taxi_duration_mllib')\
            .config("spark.driver.memory", "4g")\
            .config("spark.executor.memory", "4g")
spark = builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/12 10:02:44 WARN Utils: Your hostname, HP-Envy resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/12 10:02:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/12 10:02:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Read input datasets for training and testing:

In [3]:
#Read input files
raw_train_data = spark.read.csv('train.csv', header=True, inferSchema=True)
raw_test_data = spark.read.csv('test.csv', header=True, inferSchema=True)

## 2.Data preprocessing

Parse timestamp features in the dataset:

In [4]:
#Cast the pickup string values of training data into timestamps.
casted_train_data = raw_train_data.withColumns({
                        'pickup_datetime' : raw_train_data['pickup_datetime'].cast('timestamp'),
                    })

#Cast the pickup string values of testing data into timestamps.
casted_test_data = raw_test_data.withColumns({
                        'pickup_datetime' : raw_test_data['pickup_datetime'].cast('timestamp')
                    })

User-Defined Function (UDF)

In [5]:
# Haversine distance function
def haversine(lon1, lat1, lon2, lat2):
    if None in (lon1, lat1, lon2, lat2) or not all(isinstance(x, (int, float)) for x in [lon1, lat1, lon2, lat2]):
        return 0.0
    R = 6371  # Earth Radius (km)
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

haversine_udf = udf(haversine, DoubleType())

Extract usable features:

In [6]:
#Get usable columns from the dataframe
#Also convert timestamps into time elements and encode `store_and_fwd_flag` feature into binary values
extracted_train_df = casted_train_data.selectExpr(
        'id',
        'vendor_id',
        'YEAR(pickup_datetime)    AS pickup_year',
        'MONTH(pickup_datetime)   AS pickup_month',
        'DAY(pickup_datetime)     AS pickup_day',
        'HOUR(pickup_datetime)    AS pickup_hour',
        'MINUTE(pickup_datetime)  AS pickup_min',
        'SECOND(pickup_datetime)  AS pickup_sec',
        'passenger_count', 
        'pickup_longitude', 
        'pickup_latitude', 
        'dropoff_longitude', 
        'dropoff_latitude',
        'CASE WHEN store_and_fwd_flag == "Y" THEN 1 ELSE 0 END AS store_and_fwd_flag',
        'trip_duration'
    ).withColumn('distance_km', haversine_udf(col('pickup_longitude'), col('pickup_latitude'), col('dropoff_longitude'), col('dropoff_latitude')))

extracted_test_df = casted_test_data.selectExpr(
        'id',
        'vendor_id',
        'YEAR(pickup_datetime)    AS pickup_year',
        'MONTH(pickup_datetime)   AS pickup_month',
        'DAY(pickup_datetime)     AS pickup_day',
        'HOUR(pickup_datetime)    AS pickup_hour',
        'MINUTE(pickup_datetime)  AS pickup_min',
        'SECOND(pickup_datetime)  AS pickup_sec',
        'passenger_count', 
        'pickup_longitude', 
        'pickup_latitude', 
        'dropoff_longitude', 
        'dropoff_latitude',
        'CASE WHEN store_and_fwd_flag == "Y" THEN 1 ELSE 0 END AS store_and_fwd_flag',
    ).withColumn('distance_km', haversine_udf(col('pickup_longitude'), col('pickup_latitude'), col('dropoff_longitude'), col('dropoff_latitude')))

In [7]:
# Remove outliers
extracted_train_df = extracted_train_df.filter(
    (col('trip_duration') > 0) & 
    (col('trip_duration') < 36000) &  
    (col('passenger_count') >= 1) & 
    (col('passenger_count') <= 6) & 
    (col('distance_km') > 0) & 
    (col('distance_km') < 100)  
)

# Check for null values
print("Check for null values ​​in extracted_train_df:")
for column in extracted_train_df.columns:
    null_count = extracted_train_df.filter(col(column).isNull()).count()
    print(f"{column}: {null_count} null value")

extracted_train_df = extracted_train_df.na.drop()

Check for null values ​​in extracted_train_df:


id: 0 null value


vendor_id: 0 null value


pickup_year: 0 null value


pickup_month: 0 null value


...

pickup_latitude: 0 null value


dropoff_longitude: 0 null value


dropoff_latitude: 0 null value
store_and_fwd_flag: 0 null value


trip_duration: 0 null value


distance_km: 0 null value


Then, convert the dataset into a RDD of `LabeledPoint` objects:

In [ ]:
# Feature list (same as 3.2.1)
feature_cols = ['vendor_id', 'pickup_year', 'pickup_month', 'pickup_day', 'pickup_hour',
                'pickup_min', 'pickup_sec', 'passenger_count', 'pickup_longitude',
                'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 
                'store_and_fwd_flag', 'distance_km']

def to_labeled_point(row):
    features = [float(row[col]) for col in feature_cols]
    # mapping vendor_id: 1 -> 0, 2 -> 1
    features[0] = features[0] - 1  # vendor_id at index 0
    return LabeledPoint(row['trip_duration'], features)

train_data = extracted_train_df.rdd.map(to_labeled_point)

# Testing set conversion
def to_features_test(row):
    features = [float(row[col]) for col in feature_cols]
    features[0] = features[0] - 1  # mapping vendor_id: 1 -> 0, 2 -> 1
    return features

test_features = extracted_test_df.rdd.map(to_features_test)
test_ids = extracted_test_df.rdd.map(lambda row: row['id'])

## 3. Model training

Split data

In [9]:
# Split data into training and testing sets
# Training set proportion parameter:
train_size = 0.8

train_rdd, validation_rdd = train_data.randomSplit([train_size, 1 - train_size], seed=24) #Fixed with seed for reproductivity

# Cache RDD (Store train_rdd and validation_rdd in memory to reduce load)
train_rdd.cache()
validation_rdd.cache()

PythonRDD[197] at RDD at PythonRDD.scala:53

Fine-tuning

In [ ]:
# Use best parameters from 3.2.1
best_params = {'maxDepth': 10}

# Model training
model = DecisionTree.trainRegressor(
    train_rdd,
    categoricalFeaturesInfo={0: 2, 7: 10, 12: 2},  # vendor_id, passenger_count, store_and_fwd_flag
    impurity='variance',
    maxDepth=best_params['maxDepth']
)

25/04/12 10:03:56 WARN BlockManager: Task 86 already completed, not releasing lock for rdd_196_0


## 4. Model evaluation (hold-out)

In [14]:
predictions = model.predict(validation_rdd.map(lambda lp: lp.features))
labels_and_preds = validation_rdd.map(lambda lp: lp.label).zip(predictions)
metrics = RegressionMetrics(labels_and_preds)
rmse = metrics.rootMeanSquaredError
r2 = metrics.r2

print(f"RMSE on validation (MLlib RDD): {rmse}")
print(f"R2 on validation (MLlib RDD): {r2}")

/home/hatake/.local/lib/python3.10/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


RMSE on validation (MLlib RDD): 407.80706225247854
R2 on validation (MLlib RDD): 0.4720785680004036


## 5. Comparison with Structured API

In [15]:
structured_api_rmse = 388.07311363218764 # Số từ 3.2.1
structured_api_r2 =  0.6742563442546459  # Số từ 3.2.1
print(f"Structured API RMSE: {structured_api_rmse}")
print(f"Structured API R2: {structured_api_r2}")
print(f"RMSE Difference: {rmse - structured_api_rmse}")
print(f"R2 Difference: {r2 - structured_api_r2}")

Structured API RMSE: 388.07311363218764
Structured API R2: 0.6742563442546459
RMSE Difference: 19.733948620290903
R2 Difference: -0.20217777625424227


## 6. Prediction (test file)

In [16]:
test_predictions = model.predict(test_features)
test_predictions_df = spark.createDataFrame(
    test_ids.zip(test_predictions),
    schema=['id', 'trip_duration']
)

# Write file
test_predictions_df.coalesce(1).write.csv("prediction_mllib.csv", header=True, mode='overwrite')

test_predictions_df.show(5)

# Stop SparkSession
spark.stop()

+---------+------------------+
|       id|     trip_duration|
+---------+------------------+
|id3004672|  811.665977443609|
|id3505355| 734.6063756063757|
|id1217141| 479.7964183139044|
|id2150126|1195.0749776472092|
|id1598245| 350.4942550386137|
+---------+------------------+
only showing top 5 rows

